## ortools -cvrp

In [2]:
"""Capacited Vehicles Routing Problem (CVRP)."""
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = [
        # fmt: off
      [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
      [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
      [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
      [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
      [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
      [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
      [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
      [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
      [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
      [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
      [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
      [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
      [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
      [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
      [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
      [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
      [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0],
        # fmt: on
    ]
    data["demands"] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, ]
    data["vehicle_capacities"] = [15, 10, 20, 15]
    data["num_vehicles"] = 4
    data["depot"] = 0
    data["starts"]=[0,0,16,16]
    data["ends"]=[16,16,16,16]
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()
    display(pd.DataFrame(data["distance_matrix"]))
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["starts"],data["ends"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == "__main__":
    main()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0,548,776,696,582,274,502,194,308,194,536,502,388,354,468,776,662
1,548,0,684,308,194,502,730,354,696,742,1084,594,480,674,1016,868,1210
2,776,684,0,992,878,502,274,810,468,742,400,1278,1164,1130,788,1552,754
3,696,308,992,0,114,650,878,502,844,890,1232,514,628,822,1164,560,1358
4,582,194,878,114,0,536,764,388,730,776,1118,400,514,708,1050,674,1244
5,274,502,502,650,536,0,228,308,194,240,582,776,662,628,514,1050,708
6,502,730,274,878,764,228,0,536,194,468,354,1004,890,856,514,1278,480
7,194,354,810,502,388,308,536,0,342,388,730,468,354,320,662,742,856
8,308,696,468,844,730,194,194,342,0,274,388,810,696,662,320,1084,514
9,194,742,742,890,776,240,468,388,274,0,342,536,422,388,274,810,468


Objective: 0
Route for vehicle 0:
 0 Load(0) ->  4 Load(4) ->  3 Load(6) ->  1 Load(7) ->  16 Load(7)
Distance of the route: 0m
Load of the route: 7

Route for vehicle 1:
 0 Load(0) ->  13 Load(4) ->  10 Load(6) ->  9 Load(7) ->  5 Load(9) ->  2 Load(10) ->  16 Load(10)
Distance of the route: 0m
Load of the route: 10



IndexError: list index out of range

## shortests(nearest)

In [53]:
#import libraries
import numpy as np
import pandas as pd

# Load the distance matrix
distance_matrix = pd.read_csv("../data/final_distance.csv")
df = pd.DataFrame(distance_matrix)
dist_matrix = df.to_numpy()

def calculate_total_distance(route, dist_matrix):
    """
    Calculate the total distance of a given route using the distance matrix.
    """
    total_distance = 0
    num_points = len(route)

    for i in range(num_points - 1):
        current_node = route[i]
        next_node = route[i + 1]
        total_distance += dist_matrix[current_node, next_node]

    return total_distance

def nearest_neighbor(dist_matrix, demands, capacities):
    """
    Apply the Nearest Neighbor heuristic to find initial routes for VRP.
    Each vehicle has different capacities and routes are assigned in a round-robin fashion.
    """
    num_points = dist_matrix.shape[0]
    visited = np.zeros(num_points - 2, dtype=bool) # 모두 false로 저장
    visited = np.hstack([np.ones(1, dtype=bool), visited, np.ones(1, dtype=bool)]) # 매립지는 True로 저장하기

    routes = []
    capa = []
    num_car = len(capacities)
    car_index = 0

    while np.sum(visited) < num_points:
        current_node = 0 if car_index % num_car == 0 else len(df) - 1 # 차고지 또는 매립지로 설정
        current_capacity = 0
        route = [current_node]
        if car_index % num_car == 0:
            visited[current_node] = True # 첫번째 node0 방문

        capacity = capacities[car_index % num_car]

        while current_capacity < capacity:
            current = route[-1] # 현재노드
            nearest = None
            min_dist = float('inf') # 무한값

            for neighbor in np.where(~visited)[0]: # visit false인곳 반복
                if demands[neighbor] + current_capacity <= capacity and dist_matrix[current, neighbor] < min_dist:
                    nearest = neighbor # 가까운 노드 저장
                    min_dist = dist_matrix[current, neighbor] # 가장 가까운 거리 저장
            if nearest is None:
                break

            route.append(nearest)
            visited[nearest] = True
            current_capacity += demands[nearest]

        route.append(len(df) - 1)
        routes.append(route)
        capa.append(current_capacity)
        car_index += 1

    return routes, capa

def format_output(routes):
    """
    화살표로 노드 구현
    """
    num = 0
    for i in range(len(routes)):
        dis = calculate_total_distance(routes[i], dist_matrix)
        num += dis 
        print(f"Route{i}:{dis}")
        print(*routes[i], sep="->")
        print("")
    print("Objective:" + str(num))

def vrp_solver(distance_matrix, capacities):
    """
    Solve the VRP using the provided distance matrix and vehicle capacities.
    """
    display(distance_matrix)
    dist_matrix = df.to_numpy()
    demands = np.hstack([np.zeros(1), np.ones(len(df) - 2), np.zeros(1)]) *20
    routes, capa = nearest_neighbor(dist_matrix, demands, capacities)
    formatted_routes = format_output(routes)
    return routes, capa

# Example call to the solver with different vehicle capacities
capacities = [400, 200, 320, 100, 240]
routes, capa = vrp_solver(distance_matrix, capacities)


,0,1,2,3,4,5,6,7,8,9,...,162,163,164,165,166,167,168,169,170,171
0,0,1160,712,1258,1180,2213,902,1801,1636,1237,...,1452,982,1852,532,1006,959,514,1055,1347,5337
1,1160,0,546,432,690,1723,1105,1311,1146,747,...,962,492,1362,371,516,255,448,565,857,4913
2,712,546,0,624,763,1796,1178,1384,1219,820,...,1035,565,1435,248,589,132,256,638,930,4986
3,1258,432,624,0,1123,1532,1538,1120,955,556,...,771,626,1795,804,650,688,881,998,1290,5154
4,1180,690,763,1123,0,1629,1463,1213,949,1105,...,1320,1117,670,943,1141,827,1020,124,165,4221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,959,255,132,688,827,1630,1289,1448,1846,979,...,1134,836,812,459,811,0,287,1124,1416,5472
168,514,448,256,881,1020,1823,1482,1641,2039,1172,...,1327,1029,1005,346,1004,287,0,895,1187,5243
169,1055,565,638,998,124,1726,1385,1544,1551,1416,...,1230,932,908,981,907,1124,895,0,291,4347
170,1347,857,930,1290,165,1820,1835,1634,1321,1186,...,1193,1224,1200,1273,1199,1416,1187,291,0,4056


Route0:7792
0->22->101->16->69->161->66->72->55->168->43->48->167->58->18->1->15->28->2->165->29->171

Route1:9416
171->54->25->170->20->14->33->57->70->150->60->171

Route2:10704
171->4->169->13->51->64->56->59->151->26->19->42->152->44->163->166->41->171

Route3:9980
171->109->140->8->154->143->171

Route4:10719
171->141->110->155->156->157->144->162->108->73->36->21->91->171

Route5:11714
0->40->11->10->30->17->6->121->71->113->124->114->100->126->120->117->119->75->34->159->24->171

Route6:11043
171->145->146->142->158->147->148->153->164->160->107->171

Route7:11535
171->53->45->63->67->7->89->78->83->99->76->81->82->111->62->23->31->171

Route8:10469
171->9->136->32->37->38->171

Route9:11690
171->131->96->90->93->94->88->97->80->95->85->84->5->171

Route10:8845
0->47->35->123->39->3->49->50->12->118->105->138->68->137->134->130->61->135->128->102->132->171

Route11:14089
171->52->74->27->115->125->116->104->122->127->106->171

Route12:19304
171->139->46->77->92->87->86->103->79-

In [54]:
capa

[400.0,
 200.0,
 320.0,
 100.0,
 240.0,
 400.0,
 200.0,
 320.0,
 100.0,
 240.0,
 400.0,
 200.0,
 280.0]

In [41]:
demands=np.hstack([np.zeros(1),np.ones(len(df)),np.zeros(1)])
demands*20

array([ 0., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20.,
       20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20.,
       20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20.,
       20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20.,
       20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20.,
       20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20.,
       20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20.,
       20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20.,
       20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20.,
       20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20.,
       20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20.,
       20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20.,
       20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20.,
       20., 20., 20., 20.,  0.])

In [42]:
routes,capa=vrp_solver(df,400)

,0,1,2,3,4,5,6,7,8,9,...,162,163,164,165,166,167,168,169,170,171
0,0,1160,712,1258,1180,2213,902,1801,1636,1237,...,1452,982,1852,532,1006,959,514,1055,1347,5337
1,1160,0,546,432,690,1723,1105,1311,1146,747,...,962,492,1362,371,516,255,448,565,857,4913
2,712,546,0,624,763,1796,1178,1384,1219,820,...,1035,565,1435,248,589,132,256,638,930,4986
3,1258,432,624,0,1123,1532,1538,1120,955,556,...,771,626,1795,804,650,688,881,998,1290,5154
4,1180,690,763,1123,0,1629,1463,1213,949,1105,...,1320,1117,670,943,1141,827,1020,124,165,4221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,959,255,132,688,827,1630,1289,1448,1846,979,...,1134,836,812,459,811,0,287,1124,1416,5472
168,514,448,256,881,1020,1823,1482,1641,2039,1172,...,1327,1029,1005,346,1004,287,0,895,1187,5243
169,1055,565,638,998,124,1726,1385,1544,1551,1416,...,1230,932,908,981,907,1124,895,0,291,4347
170,1347,857,930,1290,165,1820,1835,1634,1321,1186,...,1193,1224,1200,1273,1199,1416,1187,291,0,4056


Route0:34134
0->22->101->16->69->161->66->171->54->25->170->20->14->33->171->4->169->13->51->64->56->59->151->171->109->140->8->154->171->171

Route1:34134
0->22->101->16->69->161->66->171->54->25->170->20->14->33->171->4->169->13->51->64->56->59->151->171->109->140->8->154->171->171

Route2:34134
0->22->101->16->69->161->66->171->54->25->170->20->14->33->171->4->169->13->51->64->56->59->151->171->109->140->8->154->171->171

Route3:34134
0->22->101->16->69->161->66->171->54->25->170->20->14->33->171->4->169->13->51->64->56->59->151->171->109->140->8->154->171->171

Route4:34134
0->22->101->16->69->161->66->171->54->25->170->20->14->33->171->4->169->13->51->64->56->59->151->171->109->140->8->154->171->171

Route5:41176
171->141->110->155->156->157->143->171->36->73->108->162->144->142->171->145->146->158->147->148->153->164->160->171->21->91->63->67->171->171

Route6:41176
171->141->110->155->156->157->143->171->36->73->108->162->144->142->171->145->146->158->147->148->153->164->160->17

In [43]:
capa

[150.0,
 300.0,
 500.0,
 600.0,
 600.0,
 150.0,
 300.0,
 500.0,
 600.0,
 600.0,
 150.0,
 300.0,
 500.0,
 600.0,
 600.0,
 150.0,
 300.0,
 500.0,
 600.0,
 600.0,
 150.0,
 300.0,
 500.0,
 600.0,
 600.0,
 150.0,
 300.0,
 500.0,
 600.0,
 600.0,
 150.0,
 300.0,
 500.0,
 600.0,
 600.0,
 50.0,
 50.0,
 50.0,
 50.0,
 50.0]

In [13]:
df2=pd.read_csv("../data/final_num_171.csv")
df2=df2[["x","y"]]
df_v1=df2.iloc[routes[0]+routes[6]]
df_v2=df2.iloc[routes[1]+routes[7]]
df_v3=df2.iloc[routes[2]+routes[8]]
df_v4=df2.iloc[routes[3]+routes[9]]
df_v5=df2.iloc[routes[4]+routes[10]+routes[11]]

In [14]:
num=0
route_5=[routes[0]+routes[5],routes[1]+routes[6],routes[2]+routes[7],routes[3]+routes[8]+routes[10],routes[4]+routes[9]+routes[11]]
for i in range(5):
          dis=calculate_total_distance(route_5[i], dist_matrix)
          num+=dis 
          print(f"Route{i}:{dis}")
          print(*route_5[i],sep="->")
          print("")
print("Objective:"+str(num))

#0->22->101->16->69->161->66->72->55->168->43->48->167->58->18->1->171


Route0:17920
0->22->101->16->69->161->66->72->55->168->43->48->167->58->18->1->171->171->54->25->33->57->70->14->52->27->21->91->63->67->45->53->7->171

Route1:19107
0->40->11->10->30->17->6->121->71->113->124->114->100->126->120->117->171->171->109->140->8->154->143->157->156->155->153->158->142->148->147->145->146->171

Route2:18452
0->165->28->15->44->26->151->59->56->64->51->169->13->4->170->20->171->171->141->110->76->81->82->111->62->23->31->95->85->84->5->46->106->171

Route3:33506
0->29->149->2->42->19->152->163->166->41->74->38->107->150->60->160->171->171->131->96->78->89->83->99->90->93->94->88->97->80->87->86->103->171->171->125->47->35->123->39->3->49->50->118->116->115->112->98->79->77->171

Route4:40955
0->34->75->119->159->24->37->164->32->136->9->36->73->108->162->144->171->171->128->61->135->137->134->130->132->102->138->68->105->122->104->127->12->171->171->139->92->65->133->129->171

Objective:129940
